# Libraries

In [1]:
import EDF_wrapper
import annotations_validation as av
import annotations_extraction as ae
import filters
import pyedflib
from pathlib import Path
import pandas as pd
import numpy as np
import os

In [2]:
data_root_path = "/Users/matheusnoschang/Programming/Repos/Automatic-Feature-Extraction/"
filelist_csv_path = '/Users/matheusnoschang/Programming/Repos/Automatic-Feature-Extraction/data/local_file_list.csv'

In [6]:
def extract_features_from_filelist(filelist_csv_path: str, data_root_path: str, output_path=None, signal_labels_to_extract=['BI1', 'EMG1']):
    if output_path is None:
        output_path = data_root_path + "data/features"
    files = EDF_wrapper.read_files_from_csv_filelist(data_root_path, filelist_csv_path)
    os.makedirs(output_path, exist_ok=True)
    ae.save_features_excel_files(files, csv=True, output_path=output_path,
                                    signal_labels_to_extract=signal_labels_to_extract)
    
def extract_features_from_folder(folder_path: str, output_path=None, signal_labels_to_extract=['BI1', 'EMG1']):
    if output_path is None:
        output_path = "data/features"
    files = EDF_wrapper.read_files_from_dir(folder_path)
    os.makedirs(output_path, exist_ok=True)
    ae.save_features_excel_files(files, output_path=output_path,
                                    signal_labels_to_extract=signal_labels_to_extract
                                    )

In [7]:
extract_features_from_folder('/Users/matheusnoschang/Programming/Repos/Automatic-Feature-Extraction/data/annotated', '/Users/matheusnoschang/Programming/Repos/Automatic-Feature-Extraction/data/features/folder')

The annotations dataframe is empty.
The annotations dataframe is empty.


Feature Extraction: 100%|██████████| 6/6 [00:09<00:00,  1.65s/it]


There are 2 annotations out of the pattern.
The annotations dataframe is empty.
The annotations dataframe is empty.


Feature Extraction: 100%|██████████| 6/6 [00:13<00:00,  2.33s/it]

There are 1 annotations out of the pattern.
The annotations dataframe is empty.
The annotations dataframe is empty.


...


Feature Extraction: 100%|██████████| 6/6 [00:09<00:00,  1.59s/it]

The annotations dataframe is empty.
The annotations dataframe is empty.



Feature Extraction: 100%|██████████| 3/3 [00:04<00:00,  1.63s/it]

The annotations dataframe is empty.
The annotations dataframe is empty.



Feature Extraction: 100%|██████████| 6/6 [00:06<00:00,  1.16s/it]


In [5]:
extract_features_from_filelist(filelist_csv_path, data_root_path)

The annotations dataframe is empty.
The annotations dataframe is empty.


Feature Extraction: 100%|██████████| 6/6 [00:08<00:00,  1.49s/it]


There are 1 annotations out of the pattern.
The annotations dataframe is empty.
The annotations dataframe is empty.


Feature Extraction: 100%|██████████| 9/9 [00:12<00:00,  1.43s/it]


There are 2 annotations out of the pattern.
The annotations dataframe is empty.
The annotations dataframe is empty.


...

Feature Extraction: 100%|██████████| 3/3 [00:05<00:00,  1.70s/it]

The annotations dataframe is empty.
The annotations dataframe is empty.



Feature Extraction: 100%|██████████| 6/6 [00:11<00:00,  1.92s/it]

The annotations dataframe is empty.
The annotations dataframe is empty.



Feature Extraction: 100%|██████████| 6/6 [00:08<00:00,  1.49s/it]


In [ ]:
fileslist = EDF_wrapper.read_files_from_csv_filelist(Path('data/local_file_list.csv'), load_files=True)
len(fileslist)

In [ ]:
directory = Path("data/annotated/")
files = EDF_wrapper.read_files_from_dir(directory, load_files=True)
len(files)

ae.add_swallow_annotations_to_files(fileslist, output_path="data/xlsx/filelist/")

In [ ]:
ae.save_features_excel_files(fileslist[:2], csv=True, output_path="data/xlsx/filelist/",
                                       signal_labels_to_extract=['BI1', 'EMG1'],
                                       
                                       )

In [ ]:
ae.save_directory_features_excel_files(files, output_path="data/xlsx/folderpath/",
                                       #signal_labels_to_extract=['BI1', 'EMG1']
                                       )

---

### Unit tests

In [ ]:
ann_file = files[1]  
ann_file["header"]["annotations"] = list(map(lambda x: [x[0], x[1], x[2].lower()], ann_file["header"]["annotations"]))
ann_file["header"]["annotations"]

In [ ]:
general_df = ae.create_annotations_df(ann_file, 'general')
swallows_df = ae.create_annotations_df(ann_file, 'swallows')

In [ ]:
swallows_df[swallows_df.data_label.isin(['BI1', 'EMG1'])]

In [ ]:
swallows_df.head(10)

In [ ]:
swallows_df = ae.add_basic_swallow_features(swallows_df)

In [ ]:
df = ae.add_tsfresh_features(swallows_df)
df.shape

In [ ]:
df.to_excel(f"data/xlsx/{Path(ann_file['filepath']).stem}.xlsx", index=False)

In [ ]:
def save_directory_features_excel_files(file_list, output_path="data/xlsx/", signal_labels_to_extract=None):

    dir_general_features = pd.DataFrame()
    dir_basic_swallow_features = pd.DataFrame()
    dir_tsfresh_swallow_features = pd.DataFrame()

    for edf_file in file_list:
        if av.check_annotations(edf_file):
            # Create general features excel file
            general_df = ae.create_annotations_df(edf_file, 'general', signal_labels_to_extract=signal_labels_to_extract)
            general_df_fe = ae.add_basic_general_features(general_df)
            general_df_fe = ae.add_tsfresh_features(general_df_fe)
            
            # Create baisc swallow features and tsfresh features excel files
            swallows_df = ae.create_annotations_df(edf_file, 'swallows', signal_labels_to_extract=signal_labels_to_extract)
            basic_swallows_df_fe = ae.add_basic_swallow_features(swallows_df.copy())
            #basic_swallows_df_fe = basic_swallows_df_fe
            ts_fresh_swallows_df_fe = ae.add_tsfresh_features(swallows_df)
            if general_df_fe is not None:
                dir_general_features = pd.concat([dir_general_features, general_df_fe], axis=0)
                #general_df_fe.to_excel(f"data/xlsx/{Path(edf_file['filepath']).stem}_general_features.xlsx", index=False)
            if basic_swallows_df_fe is not None:
                dir_basic_swallow_features = pd.concat([dir_basic_swallow_features, basic_swallows_df_fe], axis=0)
                #swallows_df_fe.to_excel(f"data/xlsx/{Path(edf_file['filepath']).stem}_basic_swallow_features.xlsx", index=False)
            if ts_fresh_swallows_df_fe is not None:
                dir_tsfresh_swallow_features = pd.concat([dir_tsfresh_swallow_features, ts_fresh_swallows_df_fe], axis=0)
                #swallows_df_fe.to_excel(f"data/xlsx/{Path(edf_file['filepath']).stem}_ts_fresh_swallow_features.xlsx", index=False)

    if not dir_general_features.empty:
        dir_general_features.drop(['ann_id'], axis=1, inplace=True)
        dir_general_features.to_excel(output_path + "general_features.xlsx", index=False)
    if not dir_basic_swallow_features.empty:
        dir_basic_swallow_features.drop(['data_label'], axis=1, inplace=True)
        dir_basic_swallow_features.to_excel(output_path + "basic_swallow_features.xlsx", index=False)
    if not dir_tsfresh_swallow_features.empty:
        dir_tsfresh_swallow_features.drop(['ann_id'], axis=1, inplace=True)
        dir_tsfresh_swallow_features.to_excel(output_path + "tsfresh_swallow_features.xlsx", index=False)